# Generate simulated data to link

In this case study, we imagine running a person linkage on a simulated 2030 Census Unedited File (CUF) --
see the main notebook for more details, including references used throughout this notebook.
This notebook creates simulated input (CUF) and reference files approximating what would be used in such a process,
emulating the methods found in publicly available descriptions of the Census Bureau's primary person linkage system, PVS.

In [1]:
# Query planning is now on by default, but it has some rough edges.
# See https://github.com/dask/dask/issues/10995 for general discussion
# and https://github.com/dask/dask-expr/issues/1060 for the particular
# issue I ran into.
import dask
dask.config.set({"dataframe.query-planning": False})

In [2]:
import os, time
import warnings
import datetime
# Importing pandas for access, regardless of whether we are using it as the compute engine
import pandas
import numpy
from IPython.display import display

In [3]:
%load_ext autoreload
%autoreload 1

In [4]:
%aimport vivarium_research_prl
from vivarium_research_prl import distributed_compute, utils

In [5]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
# DO NOT EDIT if this notebook is not called generate_simulated_data.ipynb!
# This notebook is designed to be run with papermill; this cell is tagged 'parameters'
data_to_use = 'small_sample'
output_dir = '../output/generate_simulated_data'
compute_engine = 'pandas'
num_jobs = 10
cpus_per_job = 2
threads_per_job = 1
memory_per_job = "10GB"
local_directory = f"/tmp/{os.environ['USER']}_{int(time.time())}_dask"

In [7]:
# Parameters
data_to_use = "small_sample"
output_dir = "output/generate_simulated_data/"
compute_engine = "pandas"


In [8]:
if compute_engine.startswith('dask'):
    utils.ensure_empty(local_directory)

In [9]:
print(datetime.datetime.now())

2024-05-22 08:50:30.040147


In [10]:
df_ops, pd = distributed_compute.start_compute_engine(
    compute_engine,
    num_jobs=num_jobs,
    cpus_per_job=cpus_per_job,
    threads_per_job=threads_per_job,
    memory_per_job=memory_per_job,
    local_directory=local_directory,
)

In [11]:
def save_file_with_ground_truth(file_name, file, ground_truth):
    # Check that file and ground truth have the same records
    assert (
        # record_id is unique
        len(file) ==
        len(df_ops.drop_duplicates(file[['record_id']]))
    )
    assert (
        len(ground_truth) ==
        len(file[['record_id']].merge(ground_truth[['record_id']], on='record_id', how='inner'))
    )

    file_path = f'{output_dir}/{data_to_use}/{file_name}.parquet'
    utils.remove_path(file_path)
    file.to_parquet(file_path)

    ground_truth_path = f'{output_dir}/{data_to_use}/{file_name}_ground_truth.parquet'
    utils.remove_path(ground_truth_path)
    ground_truth.to_parquet(ground_truth_path)

## Load pseudopeople simulated datasets

### Record ID tracking (data lineage)

We do a little bit of work here to enable tracking the "ground truth" (the simulant IDs from
pseudopeople).
We give each pseudopeople record/row a unique identifier for tracking, and then we immediately
separate the ground truth information (which we would not have if we were using real data)
from the rest of the columns (which we would have).
The ground truth is only used in the specific "ground truth" section of this notebook,
to help avoid accidentally leaking information into the case study.

Since we also combine/aggregate pseudopeople records as part of the process of generating the
simulated reference files, ground truth is a bit more complicated than you might imagine.
For example, the ground truth may tell us that a single row in a reference file is actually
a composite of several individuals, because even the deterministic linkage (by SSN) we use
here is not without error.

We handle this by tracking *all* source records used in the construction of each record in
our reference files.
This is achieved by having a table mapping composite record IDs to the "source record IDs"
(IDs of records that were directly generated by pseudopeople).
When we aggregate records, this is combined accordingly.

In [12]:
def add_unique_id_col(df, col_name='unique_id', value_prefix=''):
    if compute_engine == 'pandas' or compute_engine.startswith('modin'):
        return df.reset_index().rename(columns={'index': col_name}).assign(**{col_name: lambda df: value_prefix + df[col_name].astype(str)})
    elif compute_engine.startswith('dask'):
        # Can use cumsum as in https://stackoverflow.com/a/60852409/ if it needs
        # to be incrementing, but we just need uniqueness

        def add_id_to_partition(df_part, partition_info=None):
            return (
                df_part
                    .assign(**{col_name: range(len(df_part))})
                    .assign(**{col_name: lambda x: (
                            value_prefix +
                            str(partition_info['number'] if partition_info is not None else 0) +
                            '_' +
                            x[col_name].astype(str)
                        ).astype('large_string[pyarrow]')}
                    )
            )

        df = df.map_partitions(add_id_to_partition)

        return df_ops.ensure_large_string_capacity(df)
    else:
        raise ValueError()

def add_unique_record_id(df, dataset_name):
    return add_unique_id_col(df, col_name='record_id', value_prefix=f'{dataset_name}_')

# Initializes a table listing the pairs between record_ids and source record_ids.
# Should only be called on "source records"; that is, records that
# come directly out of pseudopeople.
def record_id_to_single_source_record_pairs(df, source_col='record_id'):
    if source_col == 'record_id':
        # We can't have duplicate column names, so we make a new column
        # literally called 'source_col'
        df = df.assign(source_col=lambda df: df[source_col])
        source_col = 'source_col'

    return df[['record_id', source_col]].rename(columns={source_col: 'source_record_id'})

In [13]:
# Operations that aggregate records, combining the source record pairs
# between all records that are aggregated

def merge_preserving_source_records(dfs, source_record_pairings, new_record_id_prefix, *args, **kwargs):
    assert len(dfs) == len(source_record_pairings)
    for df in dfs:
        assert 'record_id' in df.columns

    on = kwargs.get('on', None)
    if on is not None:
        # If there are nulls in any of the merge columns, they can't match to anything
        dfs = [df.dropna(subset=on, how='any') for df in dfs]

    result = dfs[0]
    source_record_pairs = source_record_pairings[0]
    dfs_and_source_record_pairs_to_combine = list(zip(dfs[1:], source_record_pairings[1:]))
    for index, (df_to_merge, source_record_pairs_to_merge) in enumerate(dfs_and_source_record_pairs_to_combine):
        result = (
            result.merge(df_to_merge, *args, **kwargs)
        )
        if index == len(dfs_and_source_record_pairs_to_combine) - 1:
            # Since this is the last step, these are the record_ids that will actually be returned
            accumulate_step_record_id_prefix = new_record_id_prefix
        else:
            # A dummy intermediate -- this shouldn't be exposed to the user
            accumulate_step_record_id_prefix = f'merge_iter_{index}'

        result = df_ops.rebalance(add_unique_record_id(result, accumulate_step_record_id_prefix))
        source_record_pairs = df_ops.concat([
            # The pairs that were already in result
            source_record_pairs
                .rename(columns={'record_id': 'record_id_x'})
                .merge(result[['record_id', 'record_id_x']].dropna(how='any'), on='record_id_x')
                .drop(columns=['record_id_x']),
            # The new ones
            source_record_pairs_to_merge
                .rename(columns={'record_id': 'record_id_y'})
                .merge(result[['record_id', 'record_id_y']].dropna(how='any'), on='record_id_y')
                .drop(columns=['record_id_y']),
        ])
        result = result.drop(columns=['record_id_x', 'record_id_y'])

    return result, source_record_pairs


def dedupe_preserving_source_records(df, source_record_pairs, columns_to_dedupe, new_record_id_prefix):
    result = df_ops.drop_duplicates(df[columns_to_dedupe])
    result = add_unique_record_id(result, new_record_id_prefix)
    df_to_result_mapping = (
        df[['record_id'] + columns_to_dedupe]
            .rename(columns={'record_id': 'record_id_pre_dedupe'})
            .merge(result, on=columns_to_dedupe)
            [['record_id', 'record_id_pre_dedupe']]
    )
    result_source_record_pairs = (
        source_record_pairs
            .rename(columns={'record_id': 'record_id_pre_dedupe'})
            .merge(df_to_result_mapping, on='record_id_pre_dedupe')
            .drop(columns=['record_id_pre_dedupe'])
    )
    return result, result_source_record_pairs


def concat_preserving_source_records(dfs, source_record_pairings, new_record_id_prefix):
    dfs = [df.rename(columns={'record_id': 'record_id_pre_concat'}) for df in dfs]
    result = df_ops.concat(dfs, ignore_index=True)
    result = add_unique_record_id(result, new_record_id_prefix)

    record_id_mapping = (
        result[['record_id', 'record_id_pre_concat']]
    )
    validate_kwarg = {
        'validate': 'm:1',
    } if not compute_engine.startswith('dask') else {} # Dask doesn't support validate

    all_source_record_pairings = df_ops.persist(df_ops.concat(source_record_pairings, ignore_index=False))
    result_source_record_pairs = df_ops.persist(
        all_source_record_pairings
            .rename(columns={'record_id': 'record_id_pre_concat'})
            .merge(record_id_mapping, on='record_id_pre_concat', **validate_kwarg)
            .drop(columns=['record_id_pre_concat'])
    )

    if compute_engine.startswith('dask'):
        # Manual alternative to the 'validate' kwarg
        assert len(result_source_record_pairs) == len(all_source_record_pairings)

    return result.drop(columns=['record_id_pre_concat']), result_source_record_pairs

### Custom noise types

A few noise types are really important here, but not implemented in pseudopeople, so we implement them as "add-ons."

In [14]:
def split_columns_incorrectly(df: pandas.DataFrame, columns: list[str], row_probability: float, seed: int = 1234):
    rng = numpy.random.default_rng(seed)

    to_split_incorrectly = pandas.Series(rng.random(len(df)) < row_probability, index=df.index)
    full_name = df.loc[to_split_incorrectly, columns].fillna('').agg(' '.join, axis=1).str.strip()

    num_spaces_or_hyphens = full_name.str.count('[ -]')
    split_name = full_name.str.split('[ -]', regex=True, expand=True).fillna('').values

    split_points = numpy.ceil(
        numpy.sort(rng.random((len(full_name), len(columns) - 1)), axis=1) * num_spaces_or_hyphens.values.reshape((len(full_name), 1))
    ).astype(int)
    string_start_points_1d = numpy.arange(split_name.shape[0], dtype=int) * split_name.shape[1]
    string_end_points_1d = numpy.append(string_start_points_1d[1:], [split_name.shape[0] * split_name.shape[1]])
    last_split_points_1d = None

    for split_index, column_name in enumerate(columns):
        # Map into 1D space
        new_column = split_name.copy().reshape(-1)

        if last_split_points_1d is not None:
            erase_before_slices = tuple(
                slice(string_start_point, last_split_point)
                for string_start_point, last_split_point in zip(string_start_points_1d, last_split_points_1d)
            )
        else:
            erase_before_slices = tuple()

        if split_index < split_points.shape[1]:
            split_points_1d = numpy.arange(split_name.shape[0], dtype=int) * split_name.shape[1] + split_points[:, split_index]
            erase_after_slices = tuple(
                slice(split_point, string_end_point)
                for split_point, string_end_point in zip(split_points_1d, string_end_points_1d)
            )
        else:
            erase_after_slices = tuple()

        new_column[numpy.r_[erase_before_slices + erase_after_slices]] = ''
        new_column[new_column != ''] += ' '
        df.loc[to_split_incorrectly, column_name] = (new_column.reshape(split_name.shape)).sum(axis=1)
        df.loc[to_split_incorrectly, column_name] = df.loc[to_split_incorrectly, column_name].str.strip()

        last_split_points_1d = split_points_1d

    return df

In [15]:
def shuffle_columns(df: pandas.DataFrame, columns: list[str], row_probability: float, seed: int = 1234):
    rng = numpy.random.default_rng(seed)

    to_shuffle = pandas.Series(rng.random(len(df)) < row_probability, index=df.index)
    original_values = df.loc[to_shuffle, columns].values
    random_ordering = numpy.argsort(rng.random(original_values.shape), axis=1)
    df.loc[to_shuffle, columns] = numpy.take_along_axis(original_values, random_ordering, axis=1)
    
    return df

In [16]:
def truncate_column(df: pandas.DataFrame, column: str, cell_probability: float, truncation_length: int, seed: int = 1234):
    rng = numpy.random.default_rng(seed)
    to_truncate = pandas.Series(rng.random(len(df)) < cell_probability, index=df.index)
    df.loc[to_truncate, 'last_name'] = df.loc[to_truncate, 'last_name'].str[:truncation_length]
    return df

In [17]:
def apply_custom_noise_type(df, noise_type, *args, seed: int = 1234, **kwargs):
    if compute_engine.startswith('dask'):
        return df.map_partitions(lambda partition, partition_info=None: noise_type(
            partition,
            *args,
            seed=(seed + (partition_info['number'] if partition_info is not None else 1) * 10_000),
            **kwargs,
        ), meta=df)
    else:
        return noise_type(
            df,
            *args,
            seed=seed,
            **kwargs,
        )

### 1040 tax filings

We assume that the last 5 years of taxes would be available and used in the construction of the reference files -- see section about reference files below.

Note that these are retrieved by *tax* year, so the 2029 taxes would be available in early 2030
(around when our hypothetical case study is taking place).

In [18]:
tax_years = list(range(2025, 2030))
tax_years

[2025, 2026, 2027, 2028, 2029]

In [19]:
%%time

# Combine 1040 for all years.
simulated_taxes_1040 = df_ops.concat([
    df_ops.read_parquet(
        f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_taxes_1040_{year}.parquet',
        columns=[
            'simulant_id',
            'ssn',
            'first_name',
            'middle_initial',
            'last_name',
            'mailing_address_street_number',
            'mailing_address_street_name',
            'mailing_address_unit_number',
            'mailing_address_po_box',
            'mailing_address_city',
            'mailing_address_state',
            'mailing_address_zipcode',
        ],
    )
    for year in tax_years
], ignore_index=True)

CPU times: user 90.7 ms, sys: 36 ms, total: 127 ms
Wall time: 133 ms


In [20]:
simulated_taxes_1040 = apply_custom_noise_type(
    simulated_taxes_1040,
    split_columns_incorrectly,
    ['first_name', 'middle_initial', 'last_name'],
    row_probability=0.01,
    seed=1,
)

In [21]:
# "... many of the [IRS] records contain only the first four letters of the last name."
# (Brown et al. 2023, p.30, footnote 19)
# Note that this truncation only matters for ITIN PIKing since for SSNs that are present in SSA we use name from SSA.
PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION = 0.4 # is this a good guess at "many" in the quote above?
simulated_taxes_1040 = apply_custom_noise_type(
    simulated_taxes_1040,
    truncate_column,
    'last_name',
    cell_probability=PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION,
    truncation_length=4,
    seed=2,
)

In [22]:
simulated_taxes_1040 = apply_custom_noise_type(
    simulated_taxes_1040,
    shuffle_columns,
    ['first_name', 'middle_initial', 'last_name'],
    row_probability=0.01,
    seed=3,
)

In [23]:
simulated_taxes_1040 = df_ops.persist(add_unique_record_id(simulated_taxes_1040, 'simulated_1040'))
simulated_taxes_1040_source_record_pairs = df_ops.persist(record_id_to_single_source_record_pairs(simulated_taxes_1040))

In [24]:
simulated_taxes_1040_ground_truth = simulated_taxes_1040[['record_id', 'simulant_id']]
simulated_taxes_1040 = simulated_taxes_1040.drop(columns=['simulant_id'])
simulated_taxes_1040, simulated_taxes_1040_ground_truth = df_ops.persist(simulated_taxes_1040, simulated_taxes_1040_ground_truth)
simulated_taxes_1040

,record_id,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
0,simulated_1040_0,794-23-1522,Heather,J,Cast,None,None,None,14011,Anytown,WA,00000
1,simulated_1040_1,004-53-9627,Bailey,J,Howa,1934,cypress ave,None,None,Anytown,WA,00000
2,simulated_1040_2,273-52-7750,Heather,S,Kingsley,57,shadowood dr,None,None,Anytown,WA,00000
3,simulated_1040_3,384-45-8523,Candy,S,Morr,10881,maple av,None,None,Anytown,WA,00000
4,simulated_1040_4,858-84-4517,Patrick,J,Martinez Lopez,103,71st st w,None,None,Anytown,WA,00000
...,...,...,...,...,...,...,...,...,...,...,...,...
39114,simulated_1040_39114,595-04-6651,Akari,L,Jordan,2921,curtis la,None,None,Anytown,WA,00000
39115,simulated_1040_39115,523-63-9324,Jada,T,Fern,1920,white ibis ct,None,None,Anytown,WA,00000
39116,simulated_1040_39116,187-01-0690,Bianca,L,Alred,12613,teppert,None,None,Anytown,WA,00000
39117,simulated_1040_39117,808-04-3639,Ashton,N,Coll,116,wakema rd,None,None,Anytown,WA,00000


### W2/1099 tax filings

We assume that the last 5 years of taxes would be available and used in the construction of the reference files --
see section about reference files below.

Note that these are retrieved by *tax* year, so the 2029 taxes would be available in early 2030
(around when our hypothetical case study is taking place).

In [25]:
%%time

# Combine W2/1099 for all years.
simulated_w2_1099 = df_ops.concat([
    df_ops.read_parquet(
        f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_taxes_w2_and_1099_{year}.parquet',
        columns=[
            'simulant_id',
            'ssn',
            'first_name',
            'middle_initial',
            'last_name',
            'mailing_address_street_number',
            'mailing_address_street_name',
            'mailing_address_unit_number',
            'mailing_address_po_box',
            'mailing_address_city',
            'mailing_address_state',
            'mailing_address_zipcode',
        ],
    )
    for year in tax_years
], ignore_index=True)

CPU times: user 105 ms, sys: 15.5 ms, total: 121 ms
Wall time: 121 ms


In [26]:
simulated_w2_1099 = apply_custom_noise_type(
    simulated_w2_1099,
    split_columns_incorrectly,
    ['first_name', 'middle_initial', 'last_name'],
    row_probability=0.01,
    seed=4,
)

In [27]:
simulated_w2_1099 = apply_custom_noise_type(
    simulated_w2_1099,
    truncate_column,
    'last_name',
    cell_probability=PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION,
    truncation_length=4,
    seed=5,
)

In [28]:
simulated_w2_1099 = apply_custom_noise_type(
    simulated_w2_1099,
    shuffle_columns,
    ['first_name', 'middle_initial', 'last_name'],
    row_probability=0.01,
    seed=6
)

In [29]:
simulated_w2_1099 = df_ops.persist(add_unique_record_id(simulated_w2_1099, 'simulated_w2_1099'))
simulated_w2_1099_source_record_pairs = df_ops.persist(record_id_to_single_source_record_pairs(simulated_w2_1099))

In [30]:
simulated_w2_1099_ground_truth = simulated_w2_1099[['record_id', 'simulant_id']]
simulated_w2_1099 = simulated_w2_1099.drop(columns=['simulant_id'])
simulated_w2_1099, simulated_w2_1099_ground_truth = df_ops.persist(simulated_w2_1099, simulated_w2_1099_ground_truth)
simulated_w2_1099

,record_id,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
0,simulated_w2_1099_0,854-13-6295,Erin,R,No,1501,interlake ave n,None,None,Anytown,WA,00000
1,simulated_w2_1099_1,854-13-6295,Erkn,R,Alonxi Tellez,1501,interlake ave n,None,None,Anytown,WA,00000
2,simulated_w2_1099_2,794-23-1522,Heather,J,Castillo,None,None,None,14011,Anytown,WA,00000
3,simulated_w2_1099_3,093-85-3532,Melisa,D,Rodr,947,broomfield ln,None,None,Anytown,WA,00000
4,simulated_w2_1099_4,093-85-3512,Melisa,D,Rodr,927,broomfield ln,None,None,Anytown,WA,00000
...,...,...,...,...,...,...,...,...,...,...,...,...
53637,simulated_w2_1099_53637,119-02-0252,Paul,None,Pyle,3515,javello rd,None,None,Anytown,WA,00000
53638,simulated_w2_1099_53638,079-75-0734,Lori,T,Gete,9358,e 58th st,None,None,Anytown,WA,00000
53639,simulated_w2_1099_53639,079-75-0134,Lori,None,Geter,9358,e 58th st,None,None,Anytown,WA,00000
53640,simulated_w2_1099_53640,287-19-2431,Patricia,C,Spaeth,None,rdg park dr,None,None,Xgyyowh,WA,00000


In [31]:
simulated_taxes, simulated_taxes_source_record_pairs = concat_preserving_source_records(
    [simulated_taxes_1040, simulated_w2_1099],
    [simulated_taxes_1040_source_record_pairs, simulated_w2_1099_source_record_pairs],
    new_record_id_prefix='simulated_taxes',
)

In [32]:
simulated_taxes_1040_with_itins = simulated_taxes_1040[simulated_taxes_1040.ssn.notnull() & simulated_taxes_1040.ssn.str.startswith('9')]
simulated_taxes_1040_with_itins_source_record_pairs = simulated_taxes_1040_source_record_pairs.merge(simulated_taxes_1040_with_itins[['record_id']], on='record_id', how='inner')
simulated_taxes_1040_with_itins, simulated_taxes_1040_with_itins_source_record_pairs = df_ops.persist(simulated_taxes_1040_with_itins, simulated_taxes_1040_with_itins_source_record_pairs)

%xdel simulated_taxes_1040
%xdel simulated_taxes_1040_source_record_pairs

In [33]:
%xdel simulated_w2_1099
%xdel simulated_w2_1099_source_record_pairs

In [34]:
simulated_taxes, simulated_taxes_source_record_pairs = df_ops.persist(simulated_taxes, simulated_taxes_source_record_pairs)

### 2030 Census Unedited File (CUF)

For now, we gloss over the data schema for addresses.
We don't know how addresses would be formatted in the CUF (and it's hard to guess, because
address is not part of the Census form), but it likely would have some of these fields
(street number, street name, etc) combined.

While PVS input files do not in general have names split into first, middle, and last,
I am guessing the CUF **would** have first name, middle initial, last name (which is how pseudopeople
generates it), because that [matches the Census questionnaire](https://www2.census.gov/programs-surveys/decennial/2020/technical-documentation/questionnaires-and-instructions/questionnaires/2020-informational-questionnaire-english_DI-Q1.pdf).

In [35]:
%%time

simulated_census_2030 = df_ops.read_parquet(f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_census_2030.parquet')

CPU times: user 22.9 ms, sys: 12.5 ms, total: 35.5 ms
Wall time: 32.2 ms


In [36]:
simulated_census_2030 = apply_custom_noise_type(
    simulated_census_2030,
    split_columns_incorrectly,
    ['first_name', 'middle_initial', 'last_name'],
    row_probability=0.01,
    seed=7,
)

In [37]:
simulated_census_2030 = apply_custom_noise_type(
    simulated_census_2030,
    shuffle_columns,
    ['first_name', 'middle_initial', 'last_name'],
    row_probability=0.01,
    seed=8,
)

In [38]:
simulated_census_2030 = df_ops.persist(add_unique_record_id(simulated_census_2030, 'simulated_census_2030'))

In [39]:
simulated_census_2030_ground_truth = simulated_census_2030[['record_id', 'simulant_id']]
simulated_census_2030 = simulated_census_2030.drop(columns=['simulant_id'])
simulated_census_2030, simulated_census_2030_ground_truth = df_ops.persist(simulated_census_2030, simulated_census_2030_ground_truth)
simulated_census_2030

,record_id,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year
0,simulated_census_2030_0,0_8033,Gerald,R,Butler,86,11/03/1943,1130,mallory ln,None,Anytown,WA,00000,Household,Reference person,Male,Black,2030
1,simulated_census_2030_1,0_1066,April,S,Morino,33,10/23/1996,32597,delacorte dr,None,Anytown,WA,00000,Household,Other nonrelative,Female,Black,2030
2,simulated_census_2030_2,0_1066,Loretta,T,Carley,71,06/01/1958,32597,delacorte dr,None,Anytown,WA,00000,Household,Reference person,Female,White,2030
3,simulated_census_2030_3,0_2514,Sandra,Q,Runnalls,75,03/18/1954,4458,windsog pl,None,Anytown,WA,00000,Household,Reference person,Female,Multiracial or Other,2030
4,simulated_census_2030_4,0_5627,Bobby,S,Rhimpson,48,05/30/1985,None,winding trail rd,None,Anytown,WA,00000,Household,Other nonrelative,Male,White,2030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11038,simulated_census_2030_11038,0_5,Benjamin,W,Chung,15,11/26/2014,4000,nw skydrwst pkwj,None,Anytown,WA,00000,Other noninstitutional,Noninstitutionalized group quarters population,Male,NHOPI,2030
11039,simulated_census_2030_11039,0_3761,Jesse,V,Costanzo,15,04/24/2014,278,n dillon ave,None,Anytown,WA,00000,Household,Other relative,Male,White,2030
11040,simulated_census_2030_11040,0_10678,Alexis,R,Wolverton,21,08/20/2008,1439,highway 201 north,None,Anytown,WA,00000,Household,Other relative,Female,White,2030
11041,simulated_census_2030_11041,0_4,Ella,N,Garland,23,08/23/2006,69,s hwy 701,None,Anytown,WA,00000,Military,Noninstitutionalized group quarters population,Female,Multiracial or Other,2030


In [40]:
save_file_with_ground_truth('simulated_census_2030', simulated_census_2030, simulated_census_2030_ground_truth)

In [41]:
%xdel simulated_census_2030

### Simulated SSA Numident

In [42]:
%%time

simulated_ssa_numident = df_ops.read_parquet(f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_ssa_numident.parquet')

CPU times: user 29.3 ms, sys: 8.21 ms, total: 37.5 ms
Wall time: 36.8 ms


In [43]:
simulated_ssa_numident = apply_custom_noise_type(
    simulated_ssa_numident,
    split_columns_incorrectly,
    ['first_name', 'middle_name', 'last_name'],
    row_probability=0.01,
    seed=9,
)

In [44]:
simulated_ssa_numident = apply_custom_noise_type(
    simulated_ssa_numident,
    shuffle_columns,
    ['first_name', 'middle_name', 'last_name'],
    row_probability=0.01,
    seed=10,
)

In [45]:
if data_to_use == 'ri':
    # Remove simulants who were never observed in Rhode Island -- this is unrealistic to real life!
    # We only do this to create a medium-sized set of reference files for computational testing.
    simulated_ssa_numident = (
        simulated_ssa_numident
            .merge(
                df_ops.drop_duplicates(simulated_taxes_1040_ground_truth[['simulant_id']]).assign(observed_in_taxes_1040=1),
                on='simulant_id',
                how='left',
            )
            .merge(
                df_ops.drop_duplicates(simulated_w2_1099_ground_truth[['simulant_id']]).assign(observed_in_taxes_w2_1099=1),
                on='simulant_id',
                how='left',
            )
            .merge(
                df_ops.drop_duplicates(simulated_census_2030_ground_truth[['simulant_id']]).assign(observed_in_census_2030=1),
                on='simulant_id',
                how='left',
            )
            .assign(observed_ever=lambda df: (df.observed_in_taxes_1040 == 1) | (df.observed_in_taxes_w2_1099 == 1) | (df.observed_in_census_2030 == 1))
            .pipe(lambda df: df[df.observed_ever])
            .drop(columns=["observed_in_taxes_1040", "observed_in_taxes_w2_1099", "observed_in_census_2030", "observed_ever"])
    )

In [46]:
%xdel simulated_census_2030_ground_truth

In [47]:
simulated_ssa_numident = df_ops.persist(add_unique_record_id(simulated_ssa_numident, 'simulated_ssa_numident'))
simulated_ssa_numident_source_record_pairs = df_ops.persist(record_id_to_single_source_record_pairs(simulated_ssa_numident))

In [48]:
simulated_ssa_numident_ground_truth = simulated_ssa_numident[['record_id', 'simulant_id']]
simulated_ssa_numident = simulated_ssa_numident.drop(columns=['simulant_id'])
simulated_ssa_numident, simulated_ssa_numident_ground_truth = df_ops.persist(simulated_ssa_numident, simulated_ssa_numident_ground_truth)
simulated_ssa_numident

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date
0,simulated_ssa_numident_0,786-77-6454,Evelyn,Nancy,Sorrell,19191204,Female,creation,19191204
1,simulated_ssa_numident_1,688-88-6377,George,Robert,Kelly,19210616,Male,creation,19210616
2,simulated_ssa_numident_2,651-33-9561,Beatrice,Jennie,Livingston,19220113,Female,creation,19220113
3,simulated_ssa_numident_3,665-25-7858,Eura,Nadine,Hutchison,19220305,Female,creation,19220305
4,simulated_ssa_numident_4,875-10-2359,Roberta,Ruth,Mcintyre,19220306,Female,creation,19220306
...,...,...,...,...,...,...,...,...,...
19866,simulated_ssa_numident_19866,203-10-6855,Birdie,Lizzy,Dickson,None,Female,creation,20291923
19867,simulated_ssa_numident_19867,505-88-8814,Levi,Holden,Mccain,None,Male,creation,20291224
19868,simulated_ssa_numident_19868,417-18-3811,Alan,Messiah,Casas Rodriguez,20291227,Male,creation,20291227
19869,simulated_ssa_numident_19869,450-74-3175,Emmi,Everlynn,H,20291228,Female,creation,20791228


In [49]:
source_record_ground_truth = df_ops.persist(df_ops.concat([
    simulated_ssa_numident_ground_truth,
    simulated_taxes_1040_ground_truth,
    simulated_w2_1099_ground_truth,
]).rename(columns={'record_id': 'source_record_id'}))
source_record_ground_truth

,source_record_id,simulant_id
0,simulated_ssa_numident_0,0_19979
1,simulated_ssa_numident_1,0_6846
2,simulated_ssa_numident_2,0_19983
3,simulated_ssa_numident_3,0_262
4,simulated_ssa_numident_4,0_12473
...,...,...
53637,simulated_w2_1099_53637,0_8910
53638,simulated_w2_1099_53638,0_2378
53639,simulated_w2_1099_53639,0_2378
53640,simulated_w2_1099_53640,0_22909


In [50]:
%xdel simulated_taxes_1040_ground_truth
%xdel simulated_w2_1099_ground_truth

## Create reference files

> The Census Numident – all Social Security Administration (SSA) Numident SSN records are
  edited (collapsed) to produce a Census Numident file that contains “one best-data record” for
  each SSN. All variants of name information for each SSN are retained in the Alternate Name
  Numident file, while all variants of date of birth data are retained in the Alternate DOB
  Numident. The SSN-PIK crosswalk file is used to attach a corresponding unique PIK value for
  each SSN value in the Census Numident file.

### Census Numident

Luque and Wagner, p. 4:
  
> The SSA Numident file contains all transactions ever recorded against any single SSN - with each entry
representing an addition or change (such as name changes) to the SSN record. This file is edited to
create the **Census Numident**, which contains one record for each SSN. Each SSN record in the Census
Numident contains name, DOB, sex, race, place of birth, parents’ name, citizenship status and date of death information.

and in footnote 5:

> Name edits, DOB reconciliation, and race identifiers are some of the edits conducted to produce this Numident
file. **The resulting Numident file contains the most recent name and DOB data.**

We are missing quite a few columns, since they are missing in pseudopeople's SSA Numident: race, place of birth, parents' name,
citizenship status.
However, I'm pretty sure none of these are used in linking.

In [51]:
def fill_dates(df, fill_with):
    return (
        # Replace invalid dates with nans
        pd.to_datetime(df.event_date, format='%Y%m%d', errors='coerce')
            .fillna(pandas.to_datetime('2100-01-01' if fill_with == 'latest' else '1900-01-01'))
    )

def best_data_from_columns(df, columns, best_is_latest=True):
    # We don't want to throw out events with a missing/invalid date, so we'll fill them with the value *least* likely to be chosen
    # (earlier than all values if taking the latest, later than all values if taking the earliest).
    fill_with = 'earliest' if best_is_latest else 'latest'

    result = (
        df
            # Without mutating the existing date column, get one that is actually
            # a date type and can be used for sorting.
            # Note: we actually convert this to an integer for sorting purposes, because Modin was having trouble
            # sorting by it as an actual datetime
            .assign(event_date_for_sort=lambda df: fill_dates(df, fill_with=fill_with).astype(numpy.int64) // 10 ** 9)
            .dropna(subset=columns, how='all')
            .pipe(lambda df: df_ops.drop_duplicates(df, subset='ssn', sort_col='event_date_for_sort', keep=('last' if best_is_latest else 'first')))
            [['record_id', 'ssn'] + columns]
    )

    return result, record_id_to_single_source_record_pairs(result)

best_name, best_name_source_record_pairs = best_data_from_columns(
    simulated_ssa_numident,
    columns=['first_name', 'middle_name', 'last_name'],
)

best_date_of_birth, best_date_of_birth_source_record_pairs = best_data_from_columns(
    simulated_ssa_numident,
    columns=['date_of_birth'],
)

best_date_of_death, best_date_of_death_source_record_pairs = best_data_from_columns(
    simulated_ssa_numident[simulated_ssa_numident.event_type == 'death'],
    columns=['event_date'],
)
best_date_of_death = best_date_of_death.rename(columns={'event_date': 'date_of_death'})

simulated_census_numident, simulated_census_numident_source_record_pairs = df_ops.persist(merge_preserving_source_records(
    [best_name, best_date_of_birth, best_date_of_death],
    [best_name_source_record_pairs, best_date_of_birth_source_record_pairs, best_date_of_death_source_record_pairs],
    new_record_id_prefix='simulated_census_numident',
    on=['ssn'],
    how='left',
))
simulated_census_numident

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,date_of_death
0,simulated_census_numident_0,839-01-5420,Carrie,Jennifer,Villano,19800203,NaN
1,simulated_census_numident_1,025-35-6721,David,Janna,Brown-Austin,19810905,NaN
2,simulated_census_numident_2,467-24-1994,None,Emily,Parsons,19841228,NaN
3,simulated_census_numident_3,816-41-7970,Gabrielle,Margaux,Tae,19870909,NaN
4,simulated_census_numident_4,483-04-2864,Misty,Stephanie,Johnson,19711014,NaN
...,...,...,...,...,...,...,...
18464,simulated_census_numident_18464,197-32-1520,Joshua,Samuel,Calin,20040412,NaN
18465,simulated_census_numident_18465,556-73-5897,Roger,Phil,Sells,19461109,NaN
18466,simulated_census_numident_18466,044-85-2718,Yllanca,Laura,Aiong,NaN,NaN
18467,simulated_census_numident_18467,279-60-1657,Hank,German,Barnard,20220629,NaN


### Alternate Name Numident

Wagner and Layne, p. 9:

>  All variants of name information for each SSN are retained in the Alternate Name
Numident file...

In [52]:
simulated_alternate_name_numident, simulated_alternate_name_numident_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_ssa_numident,
    simulated_ssa_numident_source_record_pairs,
    columns_to_dedupe=['ssn', 'first_name', 'middle_name', 'last_name'],
    new_record_id_prefix='simulated_alternate_name_numident',
))
simulated_alternate_name_numident

,record_id,ssn,first_name,middle_name,last_name
0,simulated_alternate_name_numident_11,685-77-0916,Betty,Audrey,Fairbrother
1,simulated_alternate_name_numident_12,047-34-5906,Elaine,,Laurel Sherman
2,simulated_alternate_name_numident_15,136-85-7371,Betty,Avis,Atterberry
3,simulated_alternate_name_numident_19,017-47-7973,George,Robert,Rwareon
4,simulated_alternate_name_numident_20,726-57-2168,Josephine,Margaret,Mulhall
...,...,...,...,...,...
18937,simulated_alternate_name_numident_19866,203-10-6855,Birdie,Lizzy,Dickson
18938,simulated_alternate_name_numident_19867,505-88-8814,Levi,Holden,Mccain
18939,simulated_alternate_name_numident_19868,417-18-3811,Alan,Messiah,Casas Rodriguez
18940,simulated_alternate_name_numident_19869,450-74-3175,Emmi,Everlynn,H


In [53]:
df_ops.compute(df_ops.groupby_agg_small_groups(simulated_alternate_name_numident, by='ssn', agg_func=lambda x: x.size()).describe())

count    18469.000000
mean         1.025610
std          0.157974
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          2.000000
dtype: float64

In [54]:
# Show alternate names for the same simulated SSN
simulated_alternate_name_numident.merge(
    df_ops.groupby_agg_small_groups(simulated_alternate_name_numident[["ssn"]], by="ssn", agg_func=lambda x: x.size()).pipe(lambda x: x[x > 1]).reset_index()[['ssn']],
    on='ssn',
    how='inner'
).sort_values('ssn').tail(100)

,record_id,ssn,first_name,middle_name,last_name
328,simulated_alternate_name_numident_2681,791-62-7882,Eric,Douglas,Upshur
617,simulated_alternate_name_numident_17247,791-62-7882,Rick,Douglas,Upshur
264,simulated_alternate_name_numident_1652,793-06-7496,None,Mary,Muskett
723,simulated_alternate_name_numident_18183,793-06-7496,Nancy,Mary,Muskett
728,simulated_alternate_name_numident_18215,794-71-6508,Sharen,Mildred,Zellers
...,...,...,...,...,...
515,simulated_alternate_name_numident_16331,898-71-7849,Fay,Jean,Gallipo
348,simulated_alternate_name_numident_2971,899-07-0109,Jeffrey,Glenn,Williams
675,simulated_alternate_name_numident_17752,899-07-0109,Jeffrey,Glenn,Wkloishs
301,simulated_alternate_name_numident_2140,899-24-9400,Willie,David,None


### Alternate DOB Numident

Wagner and Layne, p. 9:

> ... while all variants of date of birth data are retained in the Alternate DOB
Numident.

In [55]:
simulated_alternate_dob_numident, simulated_alternate_dob_numident_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_ssa_numident,
    simulated_ssa_numident_source_record_pairs,
    columns_to_dedupe=['ssn', 'date_of_birth'],
    new_record_id_prefix='simulated_alternate_dob_numident',
))
simulated_alternate_dob_numident

,record_id,ssn,date_of_birth
0,simulated_alternate_dob_numident_5,420-19-3737,19220307
1,simulated_alternate_dob_numident_14,765-44-4521,None
2,simulated_alternate_dob_numident_30,075-01-3445,19240610
3,simulated_alternate_dob_numident_34,027-46-6956,19240918
4,simulated_alternate_dob_numident_41,221-08-7443,19250225
...,...,...,...
18721,simulated_alternate_dob_numident_19866,203-10-6855,None
18722,simulated_alternate_dob_numident_19867,505-88-8814,None
18723,simulated_alternate_dob_numident_19868,417-18-3811,20291227
18724,simulated_alternate_dob_numident_19869,450-74-3175,20291228


In [56]:
df_ops.compute(df_ops.groupby_agg_small_groups(simulated_alternate_dob_numident, by='ssn', agg_func=lambda x: x.size()).describe())

count    18469.000000
mean         1.013915
std          0.117142
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          2.000000
dtype: float64

In [57]:
simulated_alternate_dob_numident.merge(
    df_ops.groupby_agg_small_groups(simulated_alternate_dob_numident[["ssn"]], by="ssn", agg_func=lambda x: x.size()).pipe(lambda x: x[x > 1]).reset_index()[['ssn']],
    on='ssn',
    how='inner'
).sort_values('ssn').tail(100)

,record_id,ssn,date_of_birth
208,simulated_alternate_dob_numident_3516,752-57-7892,19621210
502,simulated_alternate_dob_numident_19750,752-57-7892,19611210
302,simulated_alternate_dob_numident_16765,752-86-2170,20010511
251,simulated_alternate_dob_numident_11691,752-86-2170,20010521
36,simulated_alternate_dob_numident_234,758-56-9805,19330714
...,...,...,...
99,simulated_alternate_dob_numident_749,898-02-5861,19420727
277,simulated_alternate_dob_numident_16331,898-71-7849,19320809
272,simulated_alternate_dob_numident_16253,898-71-7849,17320809
362,simulated_alternate_dob_numident_17752,899-07-0109,19580911


### Name/DOB Reference File

Wagner and Layne, p. 9:

> The Name and DOB Reference files are reformatted versions of the Census Numident
and includes **all possible combinations of alternate names and dates of birth, as well as
ITIN data**. All of the reference files contain SSN/ITIN and the corresponding PIK. When
an input record is linked to a reference file, the corresponding PIK is assigned. Table 1
presents the number of observations in each of the reference files.

A slightly confusing point: sometimes the Name and DOB reference files are described
as one and the same thing, and sometimes as separate.
I believe this is because **they differ only in how they are "cut" for the PVS process:**
the name reference file is cut by first and last initial,
while the DOB reference file is cut by month and day of birth.

This is described in Wagner and Layne, p.15:

> The [DOBSearch] module matches against a re-split
version of the Numident Name Reference file, splitting the data based on month and day
of birth.

Since we handle the logic of "cutting" in the linkage process itself, we generate
a single reference file here.

Note that unlike for addresses, and unlike for the pre-processing of *input* files
(as opposed to reference files), there is no explicit nickname processing/correction here.
I am fairly sure that is accurate to the PVS, which I believe assumes that nicknames
would not be present in SSA/tax records (or at least, that the real name would appear
at least once in these records).

In [58]:
simulated_name_dob_numident_records, simulated_name_dob_numident_records_source_record_pairs = df_ops.persist(merge_preserving_source_records(
    [simulated_alternate_name_numident, simulated_alternate_dob_numident],
    [simulated_alternate_name_numident_source_record_pairs, simulated_alternate_dob_numident_source_record_pairs],
    on=['ssn'],
    how='left',
    new_record_id_prefix='name_dob_numident_records',
))
simulated_name_dob_numident_records

,record_id,ssn,first_name,middle_name,last_name,date_of_birth
0,name_dob_numident_records_0,685-77-0916,Betty,Audrey,Fairbrother,19221205
1,name_dob_numident_records_1,047-34-5906,Elaine,,Laurel Sherman,19230103
2,name_dob_numident_records_2,136-85-7371,Betty,Avis,Atterberry,19230313
3,name_dob_numident_records_3,017-47-7973,George,Robert,Rwareon,19230609
4,name_dob_numident_records_4,726-57-2168,Josephine,Margaret,Mulhall,19230728
...,...,...,...,...,...,...
19282,name_dob_numident_records_19282,203-10-6855,Birdie,Lizzy,Dickson,None
19283,name_dob_numident_records_19283,505-88-8814,Levi,Holden,Mccain,None
19284,name_dob_numident_records_19284,417-18-3811,Alan,Messiah,Casas Rodriguez,20291227
19285,name_dob_numident_records_19285,450-74-3175,Emmi,Everlynn,H,20291228


In [59]:
# Show alternate name/DOB for the same simulated SSN
simulated_name_dob_numident_records.merge(
    df_ops.groupby_agg_small_groups(simulated_name_dob_numident_records[["ssn"]], by="ssn", agg_func=lambda x: x.size()).pipe(lambda x: x[x > 1]).reset_index()[['ssn']],
    on='ssn',
    how='inner',
).sort_values('ssn').tail(100)

,record_id,ssn,first_name,middle_name,last_name,date_of_birth
1064,name_dob_numident_records_17632,839-37-3165,Linda,Caryn,Rollins,19490818
1063,name_dob_numident_records_17631,839-37-3165,Linda,Caryn,Rollins,None
301,name_dob_numident_records_1106,839-37-3165,Linda,Caryn,None,None
302,name_dob_numident_records_1107,839-37-3165,Linda,Caryn,None,19490818
115,name_dob_numident_records_165,839-60-1199,Betty,None,Ausmus,19351116
...,...,...,...,...,...,...
413,name_dob_numident_records_2378,899-07-0109,Jeffrey,Glenn,Williams,19580911
412,name_dob_numident_records_2377,899-07-0109,Jeffrey,Glenn,Williams,19650509
932,name_dob_numident_records_17019,899-07-0109,Jeffrey,Glenn,Wkloishs,19650509
1070,name_dob_numident_records_17670,899-24-9400,Willie,David,Wagner,19530914


#### Incorporating people with ITINs

Individual Taxpayer Identification Numbers (ITINs) can be issued to people who are required to file
federal taxes but are not eligible for a Social Security Number.
The most common reason for this is being an undocumented immigrant and therefore not being authorized
to work in the United States.

People without SSNs used to be impossible to assign PIKs to.
In 2011 the NORC report stated (p. 38, footnote 19):

> NORC understands that the Census Bureau has undertaken an effort to enhance the PVS reference files with IRS
files that include Individual Taxpayer Identification Numbers (ITIN). For those people who are required to file a tax
return but do not have, and may not want an SSN—such as a non-U.S. citizen—the IRS issues the taxpayer an ITIN.
This enhancement to the PVS reference file may help to match more non-U.S citizens.

By 2014 (Wagner and Layne, p. 5):

> One of the key enhancements [made in recent years] increased the coverage of the reference files by
including records for persons with Individual Taxpayer Identification Numbers assigned
by the Internal Revenue Service (ITINs) to [along with?] the SSN-based Numident data. 

I have not found a specific description of how ITIN records are constructed in any of the
publicly-available sources.
This may be because it is straightforward, or because the tax data schema is confidential.
I assume that only IRS data is used, since no other data source that I am aware of would
report ITIN.

It is stated that the ITIN records are created directly from tax filings and not
from ITIN applications (Brown et al. p. 29, footnote 16), which is convenient
because the tax filing data is what we can simulate with
pseudopeople:

> The NUMIDENT provides the PII on the SSN-holder from the issuing agency (SSA), and that PII is used in SSN
verification. **For ITINs, the Census Bureau does not have access to the ITIN applications** to the issuing agency (IRS),
so name and DOB verification of ITINs is less reliable.

"Less reliable" is a bit confusing here, because as stated above when generating
the simulated tax data, IRS data should not contain date of birth at all.
Here, we have stayed true to this by omitting it entirely.

We assume that only 1040 filings would be used for this purpose; we wouldn't expect ITINs to
show up on employer-filed W-2/1099 forms.

In [60]:
# Analogous to the process of getting alternate names and dates of birth
# from SSA, we retain all versions of the name from taxes.
simulated_name_for_itins, simulated_name_for_itins_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_taxes_1040_with_itins.rename(columns={'middle_initial': 'middle_name'}),
    simulated_taxes_1040_with_itins_source_record_pairs,
    columns_to_dedupe=['ssn', 'first_name', 'middle_name', 'last_name'],
    new_record_id_prefix='simulated_name_for_itins',
))
simulated_name_for_itins

,record_id,ssn,first_name,middle_name,last_name
0,simulated_name_for_itins_76,945-76-4582,Brandon,A,Garcia Morales
1,simulated_name_for_itins_93,905-55-4311,Pate,R,Pina
2,simulated_name_for_itins_112,970-57-8403,Arthur,M,None
3,simulated_name_for_itins_133,978-95-3331,Stephen,P,Occu
4,simulated_name_for_itins_186,990-81-6797,Guerrero,Sarah,C
...,...,...,...,...,...
818,simulated_name_for_itins_39032,977-84-6791,Liam,L,Kern
819,simulated_name_for_itins_39075,997-95-0405,Joseph,D,Post
820,simulated_name_for_itins_39080,939-53-1702,Josiah,L,Andersen
821,simulated_name_for_itins_39086,993-56-2286,Larisa,A,Sali


In [61]:
df_ops.compute(df_ops.groupby_agg_small_groups(simulated_name_for_itins, by='ssn', agg_func=lambda x: x.size()).describe())

count    509.000000
mean       1.616896
std        0.661504
min        1.000000
25%        1.000000
50%        2.000000
75%        2.000000
max        4.000000
dtype: float64

In [62]:
# How many records in the simulated name/DOB numident are in the ITIN range?
# With normal pseudopeople settings, this shouldn't happen, because SSN in SSA is noiseless.
# If this number is >0, we may be adding ITIN records as more alternates to existing records.
df_ops.compute(simulated_name_dob_numident_records.ssn.str.startswith('9').sum())

0

In [63]:
simulated_name_dob_reference_file, simulated_name_dob_reference_file_source_record_pairs = df_ops.persist(concat_preserving_source_records(
    [simulated_name_dob_numident_records, simulated_name_for_itins],
    [simulated_name_dob_numident_records_source_record_pairs, simulated_name_for_itins_source_record_pairs],
    new_record_id_prefix='simulated_name_dob_reference_file',
))
simulated_name_dob_reference_file

,record_id,ssn,first_name,middle_name,last_name,date_of_birth
0,simulated_name_dob_reference_file_0,685-77-0916,Betty,Audrey,Fairbrother,19221205
1,simulated_name_dob_reference_file_1,047-34-5906,Elaine,,Laurel Sherman,19230103
2,simulated_name_dob_reference_file_2,136-85-7371,Betty,Avis,Atterberry,19230313
3,simulated_name_dob_reference_file_3,017-47-7973,George,Robert,Rwareon,19230609
4,simulated_name_dob_reference_file_4,726-57-2168,Josephine,Margaret,Mulhall,19230728
...,...,...,...,...,...,...
20105,simulated_name_dob_reference_file_20105,977-84-6791,Liam,L,Kern,NaN
20106,simulated_name_dob_reference_file_20106,997-95-0405,Joseph,D,Post,NaN
20107,simulated_name_dob_reference_file_20107,939-53-1702,Josiah,L,Andersen,NaN
20108,simulated_name_dob_reference_file_20108,993-56-2286,Larisa,A,Sali,NaN


### GeoBase Reference File

Wagner and Layne, p. 9:

> PVS creates three other sets of reference
files containing Numident data: the **GeoBase Reference File**, the Name Reference File,
and the DOB Reference file.
The GeoBase Reference File appends addresses from administrative records attached
to Numident data, including all possible combinations of alternate names and dates of
birth for SSN. Addresses from administrative records are edited and processed through
commercial software product to clean and standardize address data. ITIN data is also
incorporated into the Geobase.

Luque and Wagner, p. 5:

> Reference files contain data from the Numident file enhanced with address
data obtained from federal AR [administrative records] files.<sup>8</sup>
The reference files, thus, contain all variants of a person’s name, DOB,
and sex, as well as current and recent addresses. These reference files are
referred to as the (PVS) Geobase reference file since addresses (a geographic component)
are appended to each person record.<sup>9</sup> It is important to note that there are
multiple Geobase reference files that are created depending on the vintage of the
incoming file to be processed through PVS.

> <sup>8</sup> Namely, data from the IRS, Department of Housing and Urban Development,
several files from the Department of Health and Human Services, and Selective Service.

> <sup>9</sup> In particular, the address data is cleaned and standardized and used
to construct a variable called GEOKEY. The GEOKEY variable is constructed as a subset
of the full address, and then is appended to the Numident data to create the
PVS Geobase Reference file.

We only have IRS data to use for addresses, and specifically only W-2/1099 data,
which is a limitation of this case study.
I can't find a concrete definition of "recent" -- as noted above, we use 5 years
of IRS data.
This may be longer than the true window, but this may end up making up for
the lack of non-IRS data sources, and get us closer to a realistic number of
alternate addresses.

Also, our address data comes out of pseudopeople already parsed into address parts
like street name, etc.
For more realism, pseudopeople should output a single string that we have to (imperfectly) parse apart.

I haven't been able to find out more about what kind of "subset" the geokey is.
It is unclear to me why geokey is "interesting" since it is just derived from the
address parts.

In [64]:
address_cols = [c for c in simulated_taxes.columns if 'mailing_address' in c]

def standardize_address_part(column):
    return (
        column
            # Remove leading or trailing whitespace
            .str.strip()
            # Turn any strings of consecutive whitespace into a single space
            .str.replace('\s+', ' ', regex=True)
            # Normalize case
            .str.upper()
            # Normalize the word street as described in the example quoted above
            # In reality, there would be many rules like this
            .str.replace('\b(STREET|STR)\b', 'ST', regex=True)
            # Make sure missingness is represented consistently
            .replace('', numpy.nan)
    )

In [65]:
simulated_tax_addresses = (
    simulated_taxes
        # Can only link these to the other files if they have an SSN
        .dropna(subset=['ssn'])
        [['record_id', 'ssn'] + address_cols]
        .assign(**{c: lambda df, c=c: standardize_address_part(df[c]) for c in address_cols})
        .pipe(df_ops.ensure_large_string_capacity)
)

In [66]:
simulated_addresses_by_ssn, simulated_addresses_by_ssn_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_tax_addresses,
    simulated_taxes_source_record_pairs,
    columns_to_dedupe=['ssn'] + address_cols,
    new_record_id_prefix='addresses_by_ssn',
))
simulated_addresses_by_ssn

,record_id,ssn,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
0,addresses_by_ssn_16,192-24-265G,None,None,None,6846,ANYTOWN,WA,00000
1,addresses_by_ssn_19,606-92-2881,2327,N COLKEGE AVE,None,None,ANYTOWN,WA,00000
2,addresses_by_ssn_21,678-32-5290,None,E CALHOUN ST,None,None,ANYTOWN,WA,00000
3,addresses_by_ssn_25,327-50-2064,9706,N AVONDALE DR,None,None,ANYTOWN,WA,00000
4,addresses_by_ssn_30,862-23-3949,4150,W MISS MUFFET LN,APRT NO 41 B,None,ANYTOWN,WA,002000
...,...,...,...,...,...,...,...,...,...
31602,addresses_by_ssn_92756,119-02-0252,3515,JAVELLO RD,None,None,ANYTOWN,WA,00000
31603,addresses_by_ssn_92757,079-75-0734,9358,E 58TH ST,None,None,ANYTOWN,WA,00000
31604,addresses_by_ssn_92758,079-75-0134,9358,E 58TH ST,None,None,ANYTOWN,WA,00000
31605,addresses_by_ssn_92759,287-19-2431,None,RDG PARK DR,None,None,XGYYOWH,WA,00000


In [67]:
num_addresses = df_ops.groupby_agg_small_groups(simulated_addresses_by_ssn, by='ssn', agg_func=lambda x: x.size()).rename('size').reset_index().sort_values('size')
num_addresses

,ssn,size
32,002-51-2470,1
33,002-58-2988,1
35,002-67-3802,1
38,002-76-3G87,1
39,002-86-0128,1
...,...,...
15145,732-98-7340,7
687,034-83-5590,7
10923,531-78-0175,8
3762,186-14-1589,8


In [68]:
# Show some SSNs with a lot of address variation
df_ops.compute(pd.merge(num_addresses.tail(10), simulated_addresses_by_ssn, on='ssn', how='inner').sort_values('ssn'))

,ssn,size,record_id,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
42,034-83-5590,7,addresses_by_ssn_18016,11818,ARBOR MEADOW DRIVE,None,None,ANYTOWN,CO,00000
48,034-83-5590,7,addresses_by_ssn_86294,13322,NW 56 AVE,None,None,ANYTOWN,WA,00000
47,034-83-5590,7,addresses_by_ssn_80642,11818,ARBOR MEADOW DRIVE,None,None,ANYTOWN,WA,00000
46,034-83-5590,7,addresses_by_ssn_69913,11818,ARBOR MEADOW DRIVE,None,None,None,WA,00000
45,034-83-5590,7,addresses_by_ssn_59105,11818,ARBOR MEADOW DRIVE,None,None,ANYTOWN,IN,00000
...,...,...,...,...,...,...,...,...,...,...
18,838-29-8962,7,addresses_by_ssn_63895,4641,NORTH CARISSA LANE,None,None,ANYTOWN,WA,00000
19,838-29-8962,7,addresses_by_ssn_74702,4341,NORTH CARISSA LANE,None,None,ANYTOWN,WA,None
20,838-29-8962,7,addresses_by_ssn_85517,4341,NIRTH CARISSA LAND,None,None,ANYTOWN,WA,00000
14,838-29-8962,7,addresses_by_ssn_8144,4341,NORTH CARISSA LANE,None,None,ANYTOWN,WA,O00O0


In [69]:
# Rough estimate of how many rows we should have in our reference file, once we do this Cartesian product
df_ops.compute(
    len(simulated_name_dob_reference_file) *
    df_ops.groupby_agg_small_groups(simulated_addresses_by_ssn, by='ssn', agg_func=lambda x: x.size()).mean()
)

32969.38482286426

In [70]:
simulated_geobase_reference_file, simulated_geobase_reference_file_source_record_pairs = df_ops.persist(merge_preserving_source_records(
    [simulated_name_dob_reference_file, simulated_addresses_by_ssn],
    [simulated_name_dob_reference_file_source_record_pairs, simulated_addresses_by_ssn_source_record_pairs],
    on=['ssn'],
    how='left',
    new_record_id_prefix='simulated_geobase_reference_file',
))
simulated_geobase_reference_file

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
0,simulated_geobase_reference_file_0,685-77-0916,Betty,Audrey,Fairbrother,19221205,47461,W ROXBURY DR,None,None,ANYTOWN,WA,00000
1,simulated_geobase_reference_file_1,047-34-5906,Elaine,,Laurel Sherman,19230103,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,simulated_geobase_reference_file_2,136-85-7371,Betty,Avis,Atterberry,19230313,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,simulated_geobase_reference_file_3,017-47-7973,George,Robert,Rwareon,19230609,6812,WASHINGTON ST,None,None,ANYTOWN,WA,00000
4,simulated_geobase_reference_file_4,726-57-2168,Josephine,Margaret,Mulhall,19230728,3743,MESA VERDE ST,None,None,ANYTOWN,WA,00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32946,simulated_geobase_reference_file_32946,939-53-1702,Josiah,L,Andersen,NaN,2904,YACHT CLUB PT,None,None,ANYTOWN,WA,00000
32947,simulated_geobase_reference_file_32947,939-53-1702,Josiah,L,Andersen,NaN,W14066,E VALENCIA RD,None,None,ANYTOWN,WA,00054
32948,simulated_geobase_reference_file_32948,939-53-1702,Josiah,L,Andersen,NaN,W14066,E VALENCIA RD,None,None,ANYTOWN,WA,00000
32949,simulated_geobase_reference_file_32949,993-56-2286,Larisa,A,Sali,NaN,1411,W 80TH ST,None,None,ANYTOWN,WA,00000


In [71]:
# Actual number of rows
len(simulated_geobase_reference_file)

32951

## Track ground truth for reference files

In [72]:
def get_simulants_of_source_records(source_record_pairs, filter_record_ids=None):
    if filter_record_ids is not None:
        source_record_pairs = source_record_pairs.pipe(filter_record_ids)

    result = (
        df_ops.drop_duplicates(source_record_pairs)
            .merge(source_record_ground_truth, on='source_record_id')
            [['record_id', 'simulant_id']]
            .pipe(df_ops.drop_duplicates)
    )
    result = result.merge(
        df_ops.groupby_agg_small_groups(result, by='record_id', agg_func=lambda x: x.simulant_id.nunique().rename('n_unique_simulants')).reset_index(),
        on='record_id',
        how='left',
    )
    return result

def get_ground_truth_for_records(source_record_pairs):
    result = df_ops.persist(get_simulants_of_source_records(source_record_pairs))

    result_nunique_describe = df_ops.compute(result['n_unique_simulants'].describe())

    if numpy.isclose(result_nunique_describe.loc['max'], 1):
        print('No collisions')
        return result

    print('Collisions:')
    display(result_nunique_describe)

    print('Simulated tax records with the most collisions:')
    most_collisions_record_id = result.sort_values('n_unique_simulants', ascending=False).head(1).record_id.iloc[0]
    most_collisions_source_record_ids = df_ops.compute(
        df_ops.drop_duplicates(source_record_pairs[source_record_pairs.record_id == most_collisions_record_id][['source_record_id']])
    )
    tax_record_ids = (
        most_collisions_source_record_ids
            .merge(
                df_ops.compute(df_ops.drop_duplicates(
                    simulated_taxes_source_record_pairs[
                        simulated_taxes_source_record_pairs.source_record_id.isin(most_collisions_source_record_ids.source_record_id)
                    ][['record_id', 'source_record_id']]
                )),
                on='source_record_id',
                how='inner',
            )
    )
    most_collisions_ground_truth = df_ops.compute(
        source_record_ground_truth[source_record_ground_truth.source_record_id.isin(most_collisions_source_record_ids.source_record_id)]
    )

    most_collisions_tax_filings = (
        tax_record_ids
            .merge(
                df_ops.compute(
                    simulated_taxes[simulated_taxes.record_id.isin(tax_record_ids.record_id)],
                ),
                on='record_id',
                how='left',
            )
            .merge(
                most_collisions_ground_truth,
                on='source_record_id',
                how='left',
            )
    )
    display(most_collisions_tax_filings)

    print('Simulated SSA records with the most collisions:')
    display(
        df_ops.compute(
            most_collisions_source_record_ids
                .merge(
                    df_ops.compute(
                        simulated_ssa_numident[simulated_ssa_numident.record_id.isin(most_collisions_source_record_ids.source_record_id)]
                            .rename(columns={'record_id': 'source_record_id'})
                    ),
                    on='source_record_id',
                    how='inner'
                )
                .merge(most_collisions_ground_truth, on='source_record_id', how='left')
        )
    )

    return result

### Simulated Census Numident

In [73]:
len(simulated_census_numident_source_record_pairs)

38271

In [74]:
simulated_census_numident_ground_truth = get_ground_truth_for_records(simulated_census_numident_source_record_pairs)

No collisions


In [75]:
simulated_census_numident_ground_truth

,record_id,simulant_id,n_unique_simulants
0,simulated_census_numident_21,0_19538,1
1,simulated_census_numident_291,0_22261,1
2,simulated_census_numident_298,0_23372,1
3,simulated_census_numident_311,0_22751,1
4,simulated_census_numident_349,0_5568,1
...,...,...,...
18464,simulated_census_numident_18445,0_19153,1
18465,simulated_census_numident_18451,0_22562,1
18466,simulated_census_numident_18455,0_12899,1
18467,simulated_census_numident_18456,0_21472,1


### Simulated Alternate Name Numident

In [76]:
len(simulated_alternate_name_numident_source_record_pairs)

19871

In [77]:
simulated_alternate_name_numident_ground_truth = get_ground_truth_for_records(simulated_alternate_name_numident_source_record_pairs)

No collisions


In [78]:
simulated_alternate_name_numident_ground_truth

,record_id,simulant_id,n_unique_simulants
0,simulated_alternate_name_numident_11,0_2471,1
1,simulated_alternate_name_numident_12,0_730,1
2,simulated_alternate_name_numident_15,0_1812,1
3,simulated_alternate_name_numident_19,0_7117,1
4,simulated_alternate_name_numident_20,0_4621,1
...,...,...,...
18937,simulated_alternate_name_numident_19866,0_23602,1
18938,simulated_alternate_name_numident_19867,0_23588,1
18939,simulated_alternate_name_numident_19868,0_23589,1
18940,simulated_alternate_name_numident_19869,0_23596,1


### Alternate DOB Numident

In [79]:
len(simulated_alternate_dob_numident_source_record_pairs)

19871

In [80]:
simulated_alternate_dob_numident_ground_truth = get_ground_truth_for_records(simulated_alternate_dob_numident_source_record_pairs)

No collisions


In [81]:
simulated_alternate_dob_numident_ground_truth

,record_id,simulant_id,n_unique_simulants
0,simulated_alternate_dob_numident_5,0_19941,1
1,simulated_alternate_dob_numident_14,0_1366,1
2,simulated_alternate_dob_numident_30,0_19743,1
3,simulated_alternate_dob_numident_34,0_12990,1
4,simulated_alternate_dob_numident_41,0_9541,1
...,...,...,...
18721,simulated_alternate_dob_numident_19866,0_23602,1
18722,simulated_alternate_dob_numident_19867,0_23588,1
18723,simulated_alternate_dob_numident_19868,0_23589,1
18724,simulated_alternate_dob_numident_19869,0_23596,1


### Simulated Name/DOB Reference File

In [82]:
len(simulated_name_dob_reference_file_source_record_pairs)

42568

In [83]:
simulated_name_dob_reference_file_ground_truth = get_ground_truth_for_records(simulated_name_dob_reference_file_source_record_pairs)

No collisions


In [84]:
simulated_name_dob_reference_file_ground_truth

,record_id,simulant_id,n_unique_simulants
0,simulated_name_dob_reference_file_15348,0_19941,1
1,simulated_name_dob_reference_file_15717,0_1366,1
2,simulated_name_dob_reference_file_15795,0_19743,1
3,simulated_name_dob_reference_file_11,0_12990,1
4,simulated_name_dob_reference_file_16134,0_9541,1
...,...,...,...
20105,simulated_name_dob_reference_file_20105,0_22674,1
20106,simulated_name_dob_reference_file_20106,0_21507,1
20107,simulated_name_dob_reference_file_20107,0_7328,1
20108,simulated_name_dob_reference_file_20108,0_22299,1


### Simulated GeoBase Reference File

In [85]:
len(simulated_geobase_reference_file_source_record_pairs)

160522

In [86]:
simulated_geobase_reference_file_ground_truth = get_ground_truth_for_records(simulated_geobase_reference_file_source_record_pairs)

Collisions:


count    34327.000000
mean         1.082559
std          0.287944
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          3.000000
Name: n_unique_simulants, dtype: float64

Simulated tax records with the most collisions:


,source_record_id,record_id,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,simulant_id
0,simulated_1040_21349,simulated_taxes_21349,333-36-9813,Charles,M,Ota,78405,sandcastle dr,None,None,Anytown,WA,00000,0_21744
1,simulated_w2_1099_28264,simulated_taxes_67383,333-36-9813,Franklin,J,Ota,78405,sandcastle dr,None,None,Anytown,WA,00000,0_16562
2,simulated_w2_1099_28266,simulated_taxes_67385,333-36-9813,Maria,M,Ota,78405,sandcastle dr,None,None,Anytown,WA,00000,0_16563


Simulated SSA records with the most collisions:


,source_record_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date,simulant_id
0,simulated_ssa_numident_17699,333-36-9813,Charles,Michael,Ota,19990611,Male,creation,23240389,0_21744


In [87]:
simulated_geobase_reference_file_ground_truth

,record_id,simulant_id,n_unique_simulants
0,simulated_geobase_reference_file_26802,0_1366,1
1,simulated_geobase_reference_file_39,0_5212,1
2,simulated_geobase_reference_file_27360,0_5595,1
3,simulated_geobase_reference_file_108,0_14382,1
4,simulated_geobase_reference_file_109,0_15523,1
...,...,...,...
34322,simulated_geobase_reference_file_31101,0_23239,2
34323,simulated_geobase_reference_file_31107,0_23240,2
34324,simulated_geobase_reference_file_7252,0_8910,1
34325,simulated_geobase_reference_file_4989,0_2378,1


In [88]:
%xdel simulated_taxes
%xdel simulated_taxes_source_record_pairs

### Get ground truth by SSN

In [89]:
simulated_ssn_simulant_pairs = df_ops.persist(
    df_ops.concat([
        simulated_census_numident[["record_id", "ssn"]].merge(simulated_census_numident_ground_truth, on="record_id"),
        simulated_alternate_name_numident[["record_id", "ssn"]].merge(simulated_alternate_name_numident_ground_truth, on="record_id"),
        simulated_alternate_dob_numident[["record_id", "ssn"]].merge(simulated_alternate_dob_numident_ground_truth, on="record_id"),
        simulated_name_dob_reference_file[["record_id", "ssn"]].merge(simulated_name_dob_reference_file_ground_truth, on="record_id"),
        simulated_geobase_reference_file[["record_id", "ssn"]].merge(simulated_geobase_reference_file_ground_truth, on="record_id"),
    ])
        [['ssn', 'simulant_id']]
        .pipe(df_ops.drop_duplicates)
)
simulated_ssn_simulant_pairs

,ssn,simulant_id
17,027-46-6956,0_12990
39,182-87-9730,0_5212
44,614-82-7024,0_12734
47,331-74-9303,0_7558
49,044-86-3127,0_3858
...,...,...
34320,977-84-6791,0_22674
34321,997-95-0405,0_21507
34324,939-53-1702,0_7328
34325,993-56-2286,0_22299


In [90]:
# How many simulated SSNs have different simulant IDs
# contributing to them?
df_ops.compute((df_ops.groupby_agg_small_groups(simulated_ssn_simulant_pairs, by='ssn', agg_func=lambda x: x.simulant_id.nunique()) > 1).sum())

1093

## Save results

In [91]:
files = {
    'simulated_census_numident': (simulated_census_numident, simulated_census_numident_ground_truth),
    'simulated_alternate_name_numident': (simulated_alternate_name_numident, simulated_alternate_name_numident_ground_truth),
    'simulated_alternate_dob_numident': (simulated_alternate_dob_numident, simulated_alternate_dob_numident_ground_truth),
    'simulated_geobase_reference_file': (simulated_geobase_reference_file, simulated_geobase_reference_file_ground_truth),
    'simulated_name_dob_reference_file': (simulated_name_dob_reference_file, simulated_name_dob_reference_file_ground_truth),
}

In [92]:
simulated_reference_files = [
    simulated_census_numident,
    simulated_alternate_name_numident,
    simulated_alternate_dob_numident,
    simulated_geobase_reference_file,
    simulated_name_dob_reference_file,
]
# TODO: Rename the ssn column to explicitly include itins, since this is confusing
simulated_all_ssns_itins_in_reference_files = df_ops.concat([df[["ssn"]] for df in simulated_reference_files], ignore_index=True)
simulated_ssn_to_pik = (
    simulated_all_ssns_itins_in_reference_files
        .pipe(df_ops.drop_duplicates)
        .pipe(add_unique_id_col, col_name='pik')
        [['ssn', 'pik']]
)
simulated_ssn_to_pik

,ssn,pik
0,027-46-6956,89099
1,182-87-9730,89114
2,614-82-7024,89116
3,331-74-9303,89117
4,044-86-3127,89118
...,...,...
18973,977-84-6791,109193
18974,997-95-0405,109194
18975,939-53-1702,109195
18976,993-56-2286,109196


In [93]:
simulated_pik_simulant_pairs = (
    simulated_ssn_simulant_pairs
        .merge(simulated_ssn_to_pik, on="ssn", how="inner")
        [['pik', 'simulant_id']]
)
simulated_pik_simulant_pairs

,pik,simulant_id
0,89099,0_12990
1,89114,0_5212
2,89116,0_12734
3,89117,0_7558
4,89118,0_3858
...,...,...
20117,109193,0_22674
20118,109194,0_21507
20119,109195,0_7328
20120,109196,0_22299


In [94]:
%xdel simulated_ssn_simulant_pairs

In [95]:
for file_name, (file, ground_truth) in files.items():
    file = df_ops.persist(file.merge(simulated_ssn_to_pik, on='ssn', how='left'))
    assert df_ops.compute(file.pik.notnull().all())

    save_file_with_ground_truth(file_name, file, ground_truth)

In [96]:
simulated_pik_simulant_pairs_path = f'{output_dir}/{data_to_use}/simulated_pik_simulant_pairs.parquet'
utils.remove_path(simulated_pik_simulant_pairs_path)
simulated_pik_simulant_pairs.to_parquet(simulated_pik_simulant_pairs_path)

In [97]:
print(datetime.datetime.now())

2024-05-22 08:50:44.042941
